In [1]:
import pandas as pd
import numpy as np
import tensorflow

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV, RepeatedStratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics import confusion_matrix, classification_report

seed = 2000
np.random.seed(seed)
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import BatchNormalization, Dropout, Flatten, Dense, Embedding
from tensorflow.keras.layers import LSTM, Conv1D, MaxPooling1D
from tensorflow.keras.preprocessing import sequence

from gensim.models import Word2Vec
from gensim.parsing.porter import PorterStemmer

import nltk
from nltk.tokenize import TweetTokenizer

import warnings
warnings.filterwarnings("ignore")

2023-01-04 15:11:51.760982: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
train_sub1_df = pd.read_csv('/Users/nitanshjain/Documents/Miscellaneous/SemEval/semeval2023task3/preprocessed_data/subtask1/train_subtask_1.csv')
print(train_sub1_df.shape)
train_sub1_df.head()
print(train_sub1_df.genre.value_counts())

(433, 10)
opinion      382
reporting     41
satire        10
Name: genre, dtype: int64


In [3]:
le = LabelEncoder()
train_sub1_df['genre'] = le.fit_transform(train_sub1_df['genre'])
print(train_sub1_df.genre.value_counts())
train_sub1_df.head()

0    382
1     41
2     10
Name: genre, dtype: int64


,id,genre,headlines,articles,preprocessed_headlines,pos_tags_headlines,er_tags_headlines,preprocessed_articles,pos_tags_articles,er_tags_articles
0,833042063,2,Chelsea Handler Admits She’s ‘Very Sexually At...,Far-left comedienne Chelsea Handler has admitt...,chelsea handler admit she s very sexually attr...,"[(Chelsea, 'NNP'), (Handler, 'NNP'), (admit, '...","[(Chelsea Handler, 'PERSON', 380), (Robert Mue...",far leave comedienne chelsea handler have admi...,"[(far, 'RB'), (leave, 'VB'), (comedienne, 'NNP...","[(Chelsea Handler, 'PERSON', 380), (FBI Specia..."
1,832959523,2,How Theresa May Botched\n,Those were the times…\nThe Times page 1 is of ...,how theresa may botch,"[(how, 'WRB'), (Theresa, 'NNP'), (May, 'NNP'),...","[(Theresa, 'GPE', 384), (May, 'DATE', 391)]",those be the time the times page 1 be of janua...,"[(those, 'DT'), (be, 'VBP'), (the, 'DT'), (tim...","[(Times, 'ORG', 383), (1, 'CARDINAL', 397), (J..."
2,833039623,2,Robert Mueller III Rests His Case—Dems NEVER W...,Carload of crazies headed to the White House w...,robert mueller iii rest his case dems never will,"[(Robert, 'NNP'), (Mueller, 'NNP'), (III, 'NNP...","[(Robert Mueller III, 'PERSON', 380), (Dems, '...",carload of crazy head to the white house want ...,"[(Carload, 'NNP'), (of, 'IN'), (crazy, 'JJ'), ...","[(the White House, 'ORG', 383), (Barack Obama,..."
3,833032367,2,Robert Mueller Not Recommending Any More Indic...,"But of course, this makes no difference to the...",robert mueller not recommend any more indictment,"[(Robert, 'NNP'), (Mueller, 'NNP'), (not, 'RB'...","[(Robert Mueller, 'PERSON', 380)]",but of course this make no difference to the p...,"[(but, 'CC'), (of, 'IN'), (course, 'NN'), (thi...","[(the New York Times, 'ORG', 383), (late Frida..."
4,814777937,2,The Far Right Is Trying to Co-opt the Yellow V...,"This weekend in Manchester, England, Yellow Ve...",the far right be try to co opt the yellow vests,"[(the, 'DT'), (Far, 'NNP'), (right, 'NN'), (be...",[],this weekend in manchester england yellow vest...,"[(this, 'DT'), (weekend, 'NN'), (in, 'IN'), (M...","[(this weekend, 'DATE', 391), (Manchester, 'GP..."


In [4]:
def creating_tokens(df):
    tokens = list()
    tokenizer = TweetTokenizer()
    
    for tweets in df.loc[:, 'preprocessed_articles']:
        # print(len(tokenizer.tokenize(tweets)))
        tokens.append(tokenizer.tokenize(tweets))
    
    df['tokens_articles'] = tokens
    
    porter_stemmer = PorterStemmer()
    # Get the stemmed_tokens
    df['stemmed_tokens_articles'] = [[porter_stemmer.stem(word) for word in tokens] for tokens in df['tokens_articles']]
    
    return df

In [24]:
train_sub1_df = creating_tokens(train_sub1_df)
x_tokens = train_sub1_df.stemmed_tokens_articles
x_tokens = x_tokens.to_frame().reset_index()

y = train_sub1_df.genre

OUTPUT_FOLDER = '/Users/nitanshjain/Documents/Miscellaneous/SemEval/semeval2023task3/codefiles/subtask1/word2vec/cbow/articles/'

tokens = pd.Series(train_sub1_df.stemmed_tokens_articles).values
# print(tokens)
word2vec_model_file = OUTPUT_FOLDER + 'word2vec_subtask1_' + str(200) + '.model'

w2v_model = Word2Vec(tokens, min_count=1, vector_size=200, window=5, workers=4, sg=0)
w2v_model.train(tokens, epochs=10, total_examples=len(tokens))
w2v_model.save(word2vec_model_file)

In [25]:
def create_file(create_file, model_file, x):
    sg_w2v_model = Word2Vec.load(model_file)
    
    with open(create_file, 'w+') as word2vec_file:
        for index, row in x.iterrows():
            model_vector = (np.mean([sg_w2v_model.wv[token] for token in row['stemmed_tokens_articles']], axis=0)).tolist()
            if index == 0:
                header = ",".join(str(ele) for ele in range(200))
                word2vec_file.write(header)
                word2vec_file.write("\n")
            
            if type(model_vector) is list:
                line1 = ",".join( [str(vector_element) for vector_element in model_vector] )
            else:
                line1 = ",".join([str(0) for i in range(200)])
            word2vec_file.write(line1)
            word2vec_file.write('\n')
    
    df = pd.read_csv(create_file)
    return df
        

In [26]:
word2vec_train_filename = OUTPUT_FOLDER + 'word2vec_subtask1_train_' + str(200) + '.csv'
word2vec_train_df = create_file(word2vec_train_filename, word2vec_model_file, x_tokens)
print(word2vec_train_df.shape)
word2vec_train_df.head()

(433, 200)


,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,-0.104391,0.294051,-0.330671,0.332436,0.437490,-0.107033,0.154053,0.287133,-0.122711,0.032380,...,0.054595,0.163935,-0.300657,-0.330896,0.482972,0.316609,-0.034116,-0.068062,0.069343,0.001905
1,-0.076844,0.294566,-0.316765,0.253483,0.266453,-0.048003,0.068486,0.373374,-0.019653,0.157083,...,0.038365,0.038384,-0.212235,-0.313794,0.362436,0.322668,0.008648,-0.028766,0.076952,-0.075754
2,-0.128247,0.368483,-0.232607,0.268401,0.268684,-0.027870,0.088875,0.276665,-0.110452,0.041748,...,0.019349,0.085398,-0.254029,-0.262924,0.382376,0.311221,-0.036377,-0.076312,0.048896,-0.025028
3,-0.105308,0.180978,-0.225814,0.313345,0.289122,-0.142726,0.075475,0.301697,-0.191133,-0.023765,...,0.022349,0.079781,-0.281291,-0.386098,0.353469,0.273019,-0.085067,-0.140508,0.059111,0.083265
4,-0.029369,0.172351,-0.063271,0.171114,0.097266,-0.134540,0.102781,0.344019,-0.101619,-0.029245,...,0.044568,0.075086,-0.202494,-0.254184,0.231624,0.143246,0.028001,-0.116902,0.047694,0.003704


In [16]:
from sklearn.preprocessing import MinMaxScaler
mm = MinMaxScaler()

x_scaled = pd.DataFrame(mm.fit_transform(word2vec_train_df))
cv = RepeatedStratifiedKFold(n_splits=10, n_repeats=3, random_state=seed)
x_scaled.head()

,0,1,2,3,4,5,6,7,8,9,...,190,191,192,193,194,195,196,197,198,199
0,0.408525,0.483526,0.188052,0.560580,0.844912,0.513030,0.584383,0.513977,0.541162,0.556517,...,0.436536,0.825545,0.645845,0.773944,0.594750,0.662820,0.472559,0.961352,0.391986,0.398983
1,0.434126,0.631779,0.345685,0.305060,0.515486,0.594942,0.413518,0.578948,0.757130,0.670465,...,0.557291,0.599732,0.704051,0.682404,0.504528,0.700849,0.670437,0.931432,0.597281,0.303035
2,0.301830,0.673488,0.477531,0.409799,0.530645,0.643677,0.449093,0.454695,0.618442,0.544489,...,0.447145,0.744672,0.606188,0.725657,0.534339,0.644764,0.590358,0.828598,0.559786,0.416487
3,0.394471,0.198804,0.395122,0.415195,0.598507,0.388575,0.395119,0.374080,0.422140,0.521936,...,0.404207,0.739410,0.584550,0.598221,0.457539,0.629367,0.437669,0.726639,0.439654,0.580934
4,0.608865,0.311870,0.688858,0.244546,0.345657,0.186663,0.393718,0.511941,0.499595,0.440655,...,0.600975,0.680243,0.492871,0.800949,0.443722,0.393811,0.654183,0.542009,0.537605,0.551000


# KNN

In [17]:
from sklearn.neighbors import KNeighborsClassifier
knn_params = {
    'n_neighbors':range(1,15),
    'weights':['uniform', 'distance']
}

knn = KNeighborsClassifier()
clf = GridSearchCV(knn, knn_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8899224806201552
Best Parameters {'n_neighbors': 6, 'weights': 'uniform'}


# Logistic Regression

In [18]:
from sklearn.linear_model import LogisticRegression
lr_params = {
    'C':np.logspace(-2,2,7), 
    'penalty':['l1', 'l2', 'elasticnet', None]
}

lr = LogisticRegression()
clf = GridSearchCV(lr, lr_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8884249471458776
Best Parameters {'C': 0.21544346900318834, 'penalty': 'l2'}


# Decison Tree

In [19]:
from sklearn.tree import DecisionTreeClassifier

dt_params = {
    'max_depth':range(2,15),
    'criterion':['gini', 'entropy', 'log_loss'],
    'splitter':['best', 'random']
}

dt = DecisionTreeClassifier()
clf = GridSearchCV(dt, dt_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8807258632840029
Best Parameters {'criterion': 'entropy', 'max_depth': 3, 'splitter': 'best'}


# Random Forest Classifier

In [20]:
from sklearn.ensemble import RandomForestClassifier

rfc_params = {
    'max_depth':range(3,15),
    'criterion':['gini', 'entropy', 'log_loss'],
}

rfc = RandomForestClassifier()
clf = GridSearchCV(rfc, rfc_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8914728682170543
Best Parameters {'criterion': 'entropy', 'max_depth': 12}


# Multinomial Naive Bayes

In [21]:
%%time
from sklearn.naive_bayes import MultinomialNB

mnb = MultinomialNB()
mnb.fit(x_scaled, y)

print('Train Accuracy Score', mnb.score(x_scaled, y))

Train Accuracy Score 0.8868360277136259
CPU times: user 10.9 ms, sys: 9.92 ms, total: 20.8 ms
Wall time: 23.6 ms


# SVM

In [22]:
from sklearn.svm import SVC

svc_params = {
    'kernel':['linear', 'poly', 'rbf', 'sigmoid'],
    'degree':range(2,5)
}

svc = SVC()
clf = GridSearchCV(svc, svc_params, cv=cv)
clf.fit(x_scaled, y)

print('Best Accuracy Score ', clf.best_score_)
print('Best Parameters', clf.best_params_)

Best Accuracy Score  0.8829985905567302
Best Parameters {'degree': 2, 'kernel': 'linear'}


# Ada-Boosting

In [23]:
from sklearn.ensemble import AdaBoostClassifier

ada = AdaBoostClassifier()
ada.fit(x_scaled, y)

print('Train Accuracy Score', ada.score(x_scaled, y))

Train Accuracy Score 0.7344110854503464


# LSTM

In [ ]:
x_scaled = x_scaled[:,:,None]

In [ ]:
from keras import Sequential
from keras.layers import LSTM

model_lstm = Sequential()

model_lstm.add(LSTM(64, input_shape = x_scaled.shape[1:], return_sequences = True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.8))

model_lstm.add(LSTM(32, return_sequences=True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(LSTM(16, return_sequences=True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(LSTM(8, return_sequences=True))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(LSTM(1, return_sequences=True, name='output'))
model_lstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_lstm.add(Dropout(rate=0.5))

model_lstm.add(Dense(64, activation = 'relu'))
model_lstm.add(Flatten())
model_lstm.add(Dense(1, activation='softmax'))

model_lstm.compile(optimizer='adam', loss='categorical_crossentropy', metrics = ['accuracy'])

model_lstm.summary()

2022-12-27 14:13:43.797353: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 lstm (LSTM)                 (None, 11720, 64)         16896     
                                                                 
 dropout (Dropout)           (None, 11720, 64)         0         
                                                                 
 lstm_1 (LSTM)               (None, 11720, 32)         12416     
                                                                 
 dropout_1 (Dropout)         (None, 11720, 32)         0         
                                                                 
 lstm_2 (LSTM)               (None, 11720, 16)         3136      
                                                                 
 dropout_2 (Dropout)         (None, 11720, 16)         0         
                                                                 
 lstm_3 (LSTM)               (None, 11720, 8)          8

In [ ]:
batch_size = 32

model_lstm.fit(x_scaled, y,
            batch_size=batch_size,
            epochs=10,
            shuffle=True)

Epoch 1/5
4/7 [================>.............] - ETA: 2:36 - loss: 0.6169 - accuracy: 0.0859

# CNN LSTM

In [ ]:
from keras import Sequential
from keras.layers import LSTM

model_clstm  =  Sequential()

model_clstm.add(Conv1D(32, (3), padding = 'same', activation = 'relu', input_shape = x_scaled.shape[1:]))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.8))

model_clstm.add(Conv1D(16, (3), padding = 'same', activation = 'relu', input_shape = x_scaled.shape[1:]))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.8))

model_clstm.add(Conv1D(8, (3), padding = 'same', activation = 'relu', input_shape = x_scaled.shape[1:]))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.8))

model_clstm.add(LSTM(32, return_sequences = True))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.5))

model_clstm.add(LSTM(16, return_sequences = True))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.5))

model_clstm.add(LSTM(1, return_sequences = True))
model_clstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_clstm.add(Dropout(rate = 0.5))

model_clstm.add(Dense(64, activation = 'relu'))
model_clstm.add(Flatten())
model_clstm.add(Dense(1, activation = 'softmax'))

model_clstm.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model_clstm.summary()

In [ ]:
batch_size = 32

model_clstm.fit(x_scaled, y,
            batch_size=batch_size,
            epochs=10,
            shuffle=True)

# Bi-LSTM

In [ ]:
from keras import Sequential
from keras.layers import LSTM, Embedding, Bidirectional


model_bilstm  =  Sequential()

model_bilstm.add(Bidirectional(LSTM(32, input_shape = x_scaled.shape[1:], return_sequences=True)))
model_bilstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_bilstm.add(Dropout(rate = 0.5))

model_bilstm.add(Bidirectional(LSTM(16, return_sequences=True)))
model_bilstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_bilstm.add(Dropout(rate = 0.5))

model_bilstm.add(Bidirectional(LSTM(1, return_sequences=True)))
model_bilstm.add(MaxPooling1D(pool_size = (5), padding = 'same'))
model_bilstm.add(Dropout(rate = 0.5))

model_bilstm.add(Dense(64, activation = 'relu'))
model_bilstm.add(Flatten())
model_bilstm.add(Dense(1, activation = 'softmax'))

model_bilstm.compile(optimizer = 'adam', loss = 'categorical_crossentropy', metrics = ['accuracy'])

model_bilstm.summary()

In [ ]:
batch_size = 32

model_bilstm.fit(x_scaled, y,
            batch_size=batch_size,
            epochs=10,
            shuffle=True)